# Tracking de objetos con YoloV8 y Bytetrack

In [14]:
# !pip install ultralytics==8.0.84
# !pip install Cython
# !pip install numpy
# !pip install lap


In [15]:
import os
# esto es para evitar un error en Windows: OMP: Error #15: Initializing libiomp5md.dll, but found libiomp5md.dll already initialized.
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import cv2
import numpy as np
from ultralytics.nn.autobackend import AutoBackend
from ultralytics.yolo.utils.plotting import Annotator, colors
import torch
from bytetrack.byte_tracker import BYTETracker
from ultralytics.yolo.data.dataloaders.stream_loaders import LoadImages
from ultralytics.yolo.utils.ops import non_max_suppression, scale_boxes
import time
import cv2


In [16]:

print(torch.cuda.is_available())

False


In [17]:
conf_thres = 0.25
iou_thres = 0.45
classes = None
agnostic_nms = False
max_det = 1000
line_thickness = 2
half = False
imgsz = (1280, 704)
vid_stride = 1

In [18]:
save_vid = True
video_file = "../../stream_bitrate2.mp4"


In [19]:
# Verificar si CUDA (GPU) está disponible y luego seleccionar el dispositivo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Usando dispositivo:", device)

Usando dispositivo: cpu


In [20]:
model = AutoBackend("yolov8n.pt").to(device)
model.warmup()
stride, names, pt = model.stride, model.names, model.pt


YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


In [21]:
bytetracker = BYTETracker(
    track_thresh=0.6, match_thresh=0.8, track_buffer=120, frame_rate=30
)
tracker = bytetracker

In [22]:
dataset = LoadImages(
    video_file,
    imgsz=imgsz,
    stride=stride,
    auto=pt,
    transforms=None,
    vid_stride=vid_stride,
)


In [23]:
zona_personas = (805, 483, 1101, 618)
zona_personas2 = (2, 336, 289, 435)
zona_coches = (620, 142, 1149, 428)

tiempo_espera_personas = {}
tiempo_espera_vehiculos = {}
vehiculos = [2, 3, 5, 7] #2: 'car', 3: 'motorcycle', 5: 'bus', 7: 'truck'
def interseccion_zona(bbox, zona):
    """ Verifica si la caja delimitadora (bbox) se cruza con la zona definida. """
    x1, y1, x2, y2 = bbox
    zx1, zy1, zx2, zy2 = zona
    return not (x2 < zx1 or x1 > zx2 or y2 < zy1 or y1 > zy2)

In [24]:
def dibujar_persona(im0, centro, color):
    radio_cabeza = 5
    longitud_cuerpo = 8
    longitud_brazos = 7
    longitud_piernas = 7

    centro_cabeza = (centro[0], centro[1] - radio_cabeza - 3)
    inicio_cuerpo = (centro[0], centro[1])
    fin_cuerpo = (centro[0], centro[1] + longitud_cuerpo)

    inicio_brazo_izquierdo = (centro[0], centro[1] + 2)
    fin_brazo_izquierdo = (centro[0] - longitud_brazos, centro[1] + 2)
    inicio_brazo_derecho = (centro[0], centro[1] + 2)
    fin_brazo_derecho = (centro[0] + longitud_brazos, centro[1] + 2)

    inicio_pierna_izquierda = (centro[0], centro[1] + longitud_cuerpo)
    fin_pierna_izquierda = (centro[0] - longitud_piernas // 2, centro[1] + longitud_cuerpo + longitud_piernas)
    inicio_pierna_derecha = (centro[0], centro[1] + longitud_cuerpo)
    fin_pierna_derecha = (centro[0] + longitud_piernas // 2, centro[1] + longitud_cuerpo + longitud_piernas)

    # Dibuja la cabeza
    cv2.circle(im0, centro_cabeza, radio_cabeza, color, -1)

    # Dibuja el cuerpo
    cv2.line(im0, inicio_cuerpo, fin_cuerpo, color, 2)

    # Dibuja los brazos
    cv2.line(im0, inicio_brazo_izquierdo, fin_brazo_izquierdo, color, 2)
    cv2.line(im0, inicio_brazo_derecho, fin_brazo_derecho, color, 2)

    # Dibuja las piernas
    cv2.line(im0, inicio_pierna_izquierda, fin_pierna_izquierda, color, 2)
    cv2.line(im0, inicio_pierna_derecha, fin_pierna_derecha, color, 2)


def dibujar_semaforo(im0, estado_semaforo, estado_semaforo_personas):
    alto, ancho = im0.shape[:2]
    radio = 15
    distancia_entre_luces = 40

    # Semáforo para vehículos
    x1, y1 = ancho - 120, 20  # Posición del semáforo para vehículos
    cv2.rectangle(im0, (x1 - 20, y1 - 20), (x1 + 20, y1 + 3 * distancia_entre_luces), (0, 0, 0), -1)  # Fondo negro
    color_rojo = (0, 0, 255) if estado_semaforo == "Rojo" else (0, 0, 0)
    color_amarillo = (0, 255, 255) if estado_semaforo == "Amarillo" else (0, 0, 0)
    color_verde = (0, 255, 0) if estado_semaforo == "Verde" else (0, 0, 0)
    cv2.circle(im0, (x1, y1), radio, color_rojo, -1)
    cv2.circle(im0, (x1, y1 + distancia_entre_luces), radio, color_amarillo, -1)
    cv2.circle(im0, (x1, y1 + 2 * distancia_entre_luces), radio, color_verde, -1)

    # Semáforo para personas
    x2, y2 = ancho - 60, 20  # Posición del semáforo para personas
    cv2.rectangle(im0, (x2 - 20, y2 - 20), (x2 + 20, y2 + 2 * distancia_entre_luces), (0, 0, 0), -1)  # Fondo negro
    color_rojo_personas = (0, 0, 255)
    color_verde_personas = (0, 255, 0)
    centro_rojo = (x2, y2)
    centro_verde = (x2, y2 + distancia_entre_luces)

    # Dibuja las luces y las personas
    cv2.circle(im0, centro_rojo, radio, color_rojo_personas if estado_semaforo_personas == "Rojo" else (0, 0, 0), -1)
    cv2.circle(im0, centro_verde, radio, color_verde_personas if estado_semaforo_personas == "Verde" else (0, 0, 0), -1)
    
    # Si la luz está activa, dibuja la persona
    if estado_semaforo_personas == "Rojo":
        dibujar_persona(im0, centro_rojo, (255, 255, 255))  # Dibuja una persona blanca en la luz roja
    if estado_semaforo_personas == "Verde":
        dibujar_persona(im0, centro_verde, (255, 255, 255))  # Dibuja una persona blanca en la luz verde



In [25]:
class ContadorSemaforo:
    def __init__(self):
        self.contador_personas = 0
        self.contador_vehiculos = 0
        self.tiempo_espera_personas = {}
        self.tiempo_espera_vehiculos = {}
        self.estado_semaforo = "Verde"
        self.estado_semaforo_personas = "Rojo"
        self.umbral_espera_maximo = 60

    def asignar_estado_semaforo(self):
        tiempo_total_personas = sum(self.tiempo_espera_personas.values())
        tiempo_total_vehiculos = sum(self.tiempo_espera_vehiculos.values())
        print("Tiempo de espera personas", tiempo_total_personas )
        print("Tiempo de espera vehiculos", tiempo_total_vehiculos )

        if (
            (self.contador_vehiculos > self.contador_personas and tiempo_total_vehiculos > tiempo_total_personas) or
            (self.contador_vehiculos < self.contador_personas and tiempo_total_vehiculos > tiempo_total_personas) or
            any(tiempo > self.umbral_espera_maximo for tiempo in self.tiempo_espera_vehiculos.values())
        ):
            self.estado_semaforo = "Verde"
            self.estado_semaforo_personas = "Rojo"
        elif (
            (self.contador_personas > self.contador_vehiculos and tiempo_total_personas > tiempo_total_vehiculos) or
            (self.contador_personas < self.contador_vehiculos and tiempo_total_personas > tiempo_total_vehiculos) or
            any(tiempo > self.umbral_espera_maximo for tiempo in self.tiempo_espera_personas.values())
        ):
            self.estado_semaforo = "Rojo"
            self.estado_semaforo_personas = "Verde"
        self.tiempo_espera_personas.clear()
        self.tiempo_espera_vehiculos.clear()
        return self.estado_semaforo, self.estado_semaforo_personas

In [26]:

# Crear una instancia de la clase ContadorSemaforo
contador_semaforo = ContadorSemaforo()

estado_semaforo = "Verde"
estado_semaforo_personas = "Rojo"
for frame_idx, batch in enumerate(dataset):    
    
    tiempo_actual = time.time()
    path, im, im0s, vid_cap, s = batch
    detections = np.empty((0, 5))
    # Asegurarse de que los datos de entrada (imágenes) también estén en la GPU
    im = torch.from_numpy(im).to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255.0  # 0 - 255 to 0.0 - 1.0
    im = torch.unsqueeze(im, 0)

    result = model(im)

    p = non_max_suppression(
        result, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det
    )
    contador_personas = 0  # Inicializar el contador de personas en cada frame
    contador_vehiculos = 0

    for i, det in enumerate(p):
        p, im0, _ = path, im0s.copy(), getattr(dataset, "frame", 0)

        if det is not None and len(det):
            det[:, :4] = scale_boxes(
                im.shape[2:], det[:, :4], im0.shape
            ).round()  # rescale boxes to im0 size

        track_result = tracker.update(det.cpu(), im0)

        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        # draw boxes for visualization
        if len(track_result) > 0:
            for j, (output) in enumerate(track_result):
                bbox = output[0:4]
                id = output[4]
                cls = output[5]
                conf = output[6]

                c = int(cls)  # integer class
                id = int(id)  # integer id
                
                if c == 0 and (interseccion_zona(bbox, zona_personas) or interseccion_zona(bbox, zona_personas2)):
                    contador_personas +=1
                    contador_semaforo.contador_personas = contador_personas
                     # Verificar si el objeto persona está dentro de la zona personas
                    if id not in tiempo_espera_personas:
                        tiempo_espera_personas[id] = tiempo_actual

                    tiempo_deteccion = tiempo_actual - tiempo_espera_personas[id]
                    
                    contador_semaforo.tiempo_espera_personas[id] = tiempo_deteccion
                    
                    color_especial = (0, 100, 0) # Color verde fuerte
                    
                    
                elif (c in vehiculos) and interseccion_zona(bbox, zona_coches):
                    contador_vehiculos +=1
                    contador_semaforo.contador_vehiculos = contador_vehiculos

                    if id not in tiempo_espera_vehiculos:
                        tiempo_espera_vehiculos[id] = tiempo_actual

                    tiempo_deteccion = tiempo_actual - tiempo_espera_vehiculos[id]
                    contador_semaforo.tiempo_espera_vehiculos[id] = tiempo_deteccion
                    
                    color_especial = (0, 0, 255)  
                    
                else:
                    # Si no está dentro de ninguna zona específica, usa el color predeterminado
                    if id in tiempo_espera_personas:
                        del tiempo_espera_personas[id]
                    if id in tiempo_espera_vehiculos:
                        del tiempo_espera_vehiculos[id]
                    color_especial = colors(c, True)
                    tiempo_deteccion = 0
                label = f"{id} {names[c]} {tiempo_deteccion:.2f}s" 
                annotator.box_label(bbox, label, color=color_especial)
                
               
    # Visualizar la cantidad de personas y vehículos en la imagen
    cv2.putText(im0, f"Personas: {contador_personas}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,0, 255), 2)
    cv2.putText(im0, f"Vehiculos: {contador_vehiculos}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 255), 2)
    dibujar_semaforo(im0, estado_semaforo, estado_semaforo_personas)
             
    # Stream results
    im0 = annotator.result()
    cv2.imshow(str(p), im0)
    key = cv2.waitKey(20)

    if key == 27:  # Presiona 'Esc' para salir
        break
    elif key == 114 or key == 82:  # Presiona 'R' o 'r' para realizar una acción
        estado_semaforo = "Rojo"
        estado_semaforo_personas = "Verde"
        pass
    elif key == 118 or key == 86:  # Presiona 'V' o 'v' para realizar otra acción
        estado_semaforo = "Verde"
        estado_semaforo_personas = "Rojo"
        pass
    elif key == 113 or key == 81:  # Presiona 'Q' o 'q' para mostrar y cambiar el estado del semáforo
        estado_semaforo, estado_semaforo_personas = contador_semaforo.asignar_estado_semaforo()
        
cv2.destroyAllWindows()

Tiempo de espera personas 18.044671535491943
Tiempo de espera vehiculos 13.317918539047241
Tiempo de espera personas 158.2437663078308
Tiempo de espera vehiculos 114.31564712524414
Tiempo de espera personas 121.62584042549133
Tiempo de espera vehiculos 46.41179943084717
Tiempo de espera personas 115.7093517780304
Tiempo de espera vehiculos 53.90148973464966
Tiempo de espera personas 118.20244359970093
Tiempo de espera vehiculos 57.74265146255493
Tiempo de espera personas 46.814388036727905
Tiempo de espera vehiculos 46.21819519996643
Tiempo de espera personas 48.26771640777588
Tiempo de espera vehiculos 67.37282514572144


Guardar el video

In [29]:

save_vid = True
video_file = "../../stream_bitrate2.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # codec
fps = 24  # Asumiendo 30 fps, ajusta según tu video
save_path = '../../output.mp4'
vid_writer = cv2.VideoWriter(save_path, fourcc, fps, (1280, 704))  # Ajusta la resolución según sea necesario


for frame_idx, batch in enumerate(dataset):
    tiempo_actual = time.time()
    path, im, im0s, vid_cap, s = batch
    detections = np.empty((0, 5))
    # Asegurarse de que los datos de entrada (imágenes) también estén en la GPU
    im = torch.from_numpy(im).to(device)
    im = im.half() if half else im.float()  # uint8 to fp16/32
    im /= 255.0  # 0 - 255 to 0.0 - 1.0
    im = torch.unsqueeze(im, 0)

    result = model(im)

    p = non_max_suppression(
        result, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det
    )

    for i, det in enumerate(p):
        p, im0, _ = path, im0s.copy(), getattr(dataset, "frame", 0)

        if det is not None and len(det):
            det[:, :4] = scale_boxes(
                im.shape[2:], det[:, :4], im0.shape
            ).round()  # rescale boxes to im0 size

        track_result = tracker.update(det.cpu(), im0)

        annotator = Annotator(im0, line_width=line_thickness, example=str(names))

        # draw boxes for visualization
        if len(track_result) > 0:
            for j, (output) in enumerate(track_result):
                bbox = output[0:4]
                id = output[4]
                cls = output[5]
                conf = output[6]

                c = int(cls)  # integer class
                id = int(id)  # integer id
                #label = f"{id} {names[c]} {conf:.2f}"

                if id not in tiempos_deteccion:
                    tiempos_deteccion[id] = tiempo_actual

                tiempo_deteccion = tiempo_actual - tiempos_deteccion[id]

                c = int(cls)  # integer class
                label = f"{id} {names[c]} {tiempo_deteccion:.2f}s" 

                if interseccion_zona(bbox, zona_personas):
                    color_especial = (0, 255, 0)  # Color verde, por ejemplo
                    annotator.box_label(bbox, label, color=color_especial)
                else:
                    color = colors(c, True)
                    annotator.box_label(bbox, label, color=color)

                """ 
                color = colors(c, True)
                annotator.box_label(bbox, label, color=color)
                """
                

    # Save results
    im0 = annotator.result()
    vid_writer.write(im0)



vid_writer.release()
cv2.destroyAllWindows()

NameError: name 'tiempos_deteccion' is not defined